In [ ]:

from typing import Callable, Tuple, TypeVar, cast, get_origin, get_args, Union, Any, Optional, Dict, List
from components import CallChannel, ComponentStarter

from components.two_wheels import TwoWheelsV3
from components.logger import LoggerComponent
from components.controller.bluetooth_controller import BlueToothCarControlSPP_V2
from components.controller.bluetooth_SPP_server import start_bluetooth_server_v2b
from components.gyroscope.gyroscope import AngularSpeedControlV3
from components.camera import Picamera2V2
from data_collection.data_collection import LoggerSet
from components import PitchAngularVelocityController, get_switches
from multiprocessing import Manager
import numpy as np
import datetime
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)

# from components import numpy_sample_setup
# r, w = numpy_sample_setup(['d', 'd'], [0, 0])

%load_ext autoreload
%autoreload 2

In [ ]:
loggerset = LoggerSet('../log/testsession'+str(datetime.datetime.now()), overwrite_ok=False)
#loggerset = LoggerSet('../log/session'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

In [ ]:

logger_process = ComponentStarter(
    LoggerComponent, 
    manager, 
    init_kwargs=dict(
        loggerset = loggerset
    ),
    loop_intervals={'step': 1/100},
)

two_wheel_process = ComponentStarter(
    TwoWheelsV3, 
    manager, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = TwoWheelsV3.entry

)

angular_speed_control_process = ComponentStarter(
    AngularSpeedControlV3, 
    manager, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = AngularSpeedControlV3.entry
)

bluetooth_control_process = ComponentStarter(
    BlueToothCarControlSPP_V2, 
    manager, 
    init_kwargs={},
    loop_intervals={'step': 1/100},
)

bt_ser_out, bt_ser_process_man = start_bluetooth_server_v2b(manager)


picamera_process = ComponentStarter(
    Picamera2V2, 
    manager, 
    init_kwargs=dict(
        resolution=(114, 64), 
        framerate=30,
        name='Picamera2V2'
    ),
    loop_intervals={'step': 1/30},
    sample_setup_kwargs=dict(default_values=[np.zeros((64, 114, 3), dtype=np.uint8)])
)

pitch_process = ComponentStarter(
    PitchAngularVelocityController,
    manager, 
    init_kwargs=dict(speed=100),
    loop_intervals={'step': 1/30}, 
    instantiator=PitchAngularVelocityController.entry
    

)


In [ ]:
two_wheel_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)

angular_speed_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)


bluetooth_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)
pitch_process.register_outgoing_rpc(
    dict(logger=logger_process.incoming_rpcs['log'])
)

picamera_process.register_outgoing_rpc(
    dict(
        log=logger_process.incoming_rpcs['log'],
        setup_video_saver=logger_process.incoming_rpcs['setup_video_saver'],
        save_video_frame=logger_process.incoming_rpcs['save_video_frame']
        )
)

def get_pitch_drive_switch():
    return bool(bt_ser_out[0]().get('start'))

to_imu_controller = get_switches(pitch_process.outgoing_samples, bluetooth_control_process.outgoing_samples, get_pitch_drive_switch)


two_wheel_process.register_incoming_samples(
    angular_speed_control_process.outgoing_sample_readers
)

angular_speed_control_process.register_incoming_samples(
    to_imu_controller
)


bluetooth_control_process.register_incoming_samples(
    bt_ser_out
)


In [ ]:
bt_ser_process_man.start()
bluetooth_control_process.start()
logger_process.start()
two_wheel_process.start()
angular_speed_control_process.start()
picamera_process.start()
pitch_process.start()


In [ ]:
bt_ser_process_man.kill()
bluetooth_control_process.process_starter.kill()
#logger_process.process_starter.kill()
two_wheel_process.process_starter.kill()
angular_speed_control_process.process_starter.kill()
picamera_process.process_starter.kill()
pitch_process.process_starter.kill()



In [ ]:
from utils import mon_samples
mon_samples(pitch_process.outgoing_samples)

# playground

In [ ]:
import numpy as np
import datetime
from data_collection.data_collection import Logger
import plotly.express as px


In [ ]:
sesspath = '../log/testsession2024-07-06 20:56:00.157529/'

In [ ]:
l = Logger(sesspath,name='PitchAngularVelocityController', overwrite_ok=True)
lw2 = Logger(sesspath,name='TwoWheelsV3', overwrite_ok=True)

In [ ]:
df = l.load_as_df()
dfw2 = lw2.load_as_df()
df['power_norm'] = df['power']/1e4

In [ ]:
import pandas as pd
px.line(pd.concat([df, dfw2], axis=0), 'time',['pitch', 'speed', 'left','power_norm',])

In [ ]:
px.line(pd.concat([df], axis=0), y=['pitch', 'speed','power_norm',])

In [ ]:
px.imshow(spectra, y=freqs[::-1], aspect='auto')

In [ ]:
df[df.time > pd.to_datetime("2024-07-06 17:19:17")]

In [ ]:
from scipy.fft import fft, fftfreq
v = df['sig'].dropna()
v = v.apply(fft)
v = v.apply(lambda x: x.real**2+x.imag**2)
v = v.apply(lambda x: x[:len(x)//2])
freqs = fftfreq((4000), 1/16000)[:2000]


In [ ]:
px.line(df['sig'].iloc[6])

In [ ]:
px.line(df['sig'].iloc[601])

In [ ]:
from IPython.display import Audio
Audio(df['sig'].iloc[606], rate=16000)

In [ ]:
import scipy.signal as ss

In [ ]:
px.line(ss.correlate(df['sig'].iloc[5], df['sig'].iloc[6]))

In [ ]:
px.line(df['sig'].iloc[5])

In [ ]:
-1-50:

In [ ]:
[1,2,3,4,5][-2-3:5-2]

In [ ]:
sig2[:3]

In [ ]:
from typing import List
from nb_tools import show_global_variables
@show_global_variables
def piece_signals(sigs: List[np.ndarray]):

    full_sig = sigs[0]
    for i in range(len(sigs)-1):
        try: 
            full_sig = piece_signal(full_sig, sigs[i+1])
        except:
            return full_sig
    return full_sig



In [ ]:
full_sig  = piece_signals(sigs)

In [ ]:
f = tafn.detect_pitch_frequency(tch.tensor(full_sig).to(tch.float32), 16000)

In [ ]:
full_sig.shape

In [ ]:
489325/3044

In [ ]:
len(f)

In [ ]:
px.line(y=f, x=np.arange(len(f))/160.7)

In [ ]:
px.line(y=f)

In [ ]:
Audio(full_sig, rate=16000)

In [ ]:
sigs = df['sig'].tolist()

In [ ]:
def piece_signal(sig1, sig2, overlap_length=50):
    step=0
    while True:
        #print((-step-overlap_length))
        #print((len(sig1)-step))
        sig_to_test = sig1[(-step-overlap_length):(len(sig1)-step)]
        
        if (sig2[:overlap_length] == sig_to_test ).all():
            return np.r_[sig1[:(-step-overlap_length)], sig2]
        else:
            step+=1 
        if step > len(sig1):
            break
    

piece_signal(df['sig'][901], df['sig'][902])

In [ ]:
px.line(piece_signal(df['sig'][901], df['sig'][902]))

In [ ]:
px.line(df['sig'][901])

In [ ]:
px.line(np.c_[df['sig'][902], df['sig'][901]], )

In [ ]:
px.line(piece_signal(df['sig'].iloc[5], df['sig'].iloc[6]))

In [ ]:
px.line(piece_signal(df['sig'].iloc[5], df['sig'].iloc[6]))

In [ ]:
px.line(df['sig'].iloc[6])

In [ ]:
df['sig'][1]

In [ ]:
px.line(df['sig'].iloc[606])

In [ ]:
import torchaudio as ta
import torch as tch
tafn = ta.functional

In [ ]:
sigt = tch.tensor(df['sig'].iloc[593])

In [ ]:
sigt.to(tch.float32)

In [ ]:
tafn.detect_pitch_frequency(
    sigt.to(tch.float32), 
    16000
)

In [ ]:
sigt.to(tch.float32).shape

In [ ]:
sigt = tch.tensor(df['sig'].iloc[606])
tafn.detect_pitch_frequency(
    sigt.to(tch.float32), 
    16000, 
    win_length=46, 
)

In [ ]:
px.line(df['sig'].iloc[593])

In [ ]:
px.line(df['sig'].iloc[595])

In [ ]:
px.line(df['sig'].iloc[342])

In [ ]:
px.line(df['sig'].iloc[709])

In [ ]:
px.line(x=freqs[0:],y=np.log(v.iloc[590])[0:])

In [ ]:
px.line(x=freqs[0:],y=[np.log(v.iloc[653]), np.log(v.iloc[585]) ])

In [ ]:
px.line(x=freqs[0:],y=np.log(v.iloc[585])[0:])

In [ ]:
spectra = np.log(np.stack(v.tolist())).T[::-1]

In [ ]:
px.imshow(spectra, y=freqs[::-1], aspect='auto')